In [1]:
import json
import os
import mp
import numpy as np
from collections import defaultdict
from glob import glob
from tqdm import tqdm
import soundfile as sf
import re

timestamps = [i * 0.02 for i in range(1500 + 1)]

In [2]:
from huggingface_hub import hf_hub_download
import fasttext

filename = hf_hub_download(
    repo_id="mesolitica/fasttext-language-detection-bahasa-en", 
    filename="fasttext.ftz"
)
lang_model = fasttext.load_model(filename)

In [3]:
def chunk(alignment, reject = -7, minimum_length = 1.0):
    alls, temp = [], []
    for a in alignment:
        if a['score'] <= reject:
            if len(temp):
                if (temp[-1]['end'] - temp[0]['start']) >= minimum_length:
                    temp[-1]['end'] = float(temp[-1]['end']) + 0.1
                    alls.append(temp)
                temp = []
        else:
            temp.append(a)
            
    if len(temp):
        if (temp[-1]['end'] - temp[0]['start']) >= minimum_length:
            temp[-1]['end'] = float(temp[-1]['end']) + 0.1
            alls.append(temp)
    return alls

In [4]:
data = defaultdict(list)
with open('prepared-imda.jsonl') as fopen:
    for no, l in tqdm(enumerate(fopen)):
        l = json.loads(l)
        data[l['audio_filename']].append((no, l))
        
len(data)

1861125it [00:10, 178922.05it/s]


1861082

In [5]:
rows = list(data.values())
len(rows)

1861082

In [6]:
rows[0]

[(0,
  {'new_text': '<|startoftranscript|><|en|><|transcribe|><|0.00|> Households with target sets were encouraged to try keeping their water consumption below these designated levels.<|7.00|><|endoftext|>',
   'audio_filename': 'IMDA-STT/part1-mp3/000010101.mp3'})]

In [7]:
!rm -rf /home/husein/ssd3/prepared-imda-chunks
!mkdir /home/husein/ssd3/prepared-imda-chunks

In [8]:
def loop(data):
    data, _ = data
    new_data = []
    for d in tqdm(data):
        
        aligns, scores = [], []
        for i in d:
            f = f'prepared-imda_alignment/{i[0]}.alignment'
            try:
                with open(f) as fopen:
                    align = json.load(fopen)
                    score = np.sum([s['score'] for s in align])
                    aligns.append(align)
                    scores.append(score)
            except:
                aligns.append([])
                scores.append(-9999)

        argmax = np.argmax(scores)
        no = d[argmax][0]
        text = d[argmax][1]['new_text']
        lang = text.split('<|startoftranscript|><|')[1].split('|')[0]
        cleaned_text = re.sub(r"<\|.*?\|>", "", text).strip()
        if lang_model.predict(cleaned_text)[0][0] == '__label__english':
            predict_lang = 'en'
        else:
            predict_lang = 'ms'
        
        chunks = chunk(aligns[argmax])
        audio_filename = d[argmax][1]['audio_filename'].replace('IMDA-STT/', '')
        if len(chunks):
            y, sr = sf.read(audio_filename)
            for k, c in enumerate(chunks):
                
                y_ = y[int(sr * c[0]['start']): int(sr * c[-1]['end'])]
                
                skip = False
                
                for c_ in c:
                    if (c_['end'] - c_['start']) > 2:
                        skip = True
                        break
                if skip:
                    continue
                
                for no_ in range(len(c)):
                    if no_ > 0 and (c[no_]['start'] - c[no_ - 1]['end']) > 1.2:
                        skip = True
                        break
                if skip:
                    continue
                    
                ts = []
                
                min_t = min([c_['start'] for c_ in c])
                
                for c_ in c:
                    start = min(timestamps, key=lambda t: abs(t - (c_['start'] - min_t)))
                    end = min(timestamps, key=lambda t: abs(t - (c_['end'] - min_t)))
                    w = c_['text']
                    t = f"<|{start:.2f}|> {w}<|{end:.2f}|>"
                    ts.append(t)
                    
                new_f = os.path.join('/home/husein/ssd3/prepared-imda-chunks', f'{no}-{k}.mp3')
                if not os.path.exists(new_f):
                    sf.write(new_f, y_, sr)
                    
                word = ''.join(ts)
                word = f"<|startoftranscript|><|{predict_lang}|><|transcribeprecise|>{word}<|endoftext|>"
                
                segments, temp = [], [c[0]]
                last_t = c[0]['end']
                for c_ in c[1:]:
                    if (c_['start'] - last_t) > 0.25:
                        segments.append(temp)
                        temp = []

                    last_t = c_['end']
                    temp.append(c_)

                if len(temp):
                    segments.append(temp)
                
                ts = []
                for s in segments:
                    start = min(timestamps, key=lambda t: abs(t - (s[0]['start'] - min_t)))
                    end = min(timestamps, key=lambda t: abs(t - (s[-1]['end'] - min_t)))
                    w = ' '.join([c_['text'] for c_ in s])
                    t = f"<|{start:.2f}|> {w}<|{end:.2f}|>"
                    ts.append(t)
                
                ts = ''.join(ts)
                new_text = f"<|startoftranscript|><|{predict_lang}|><|transcribe|>{ts}<|endoftext|>"
                
                new_data.append({
                    'audio_filename': new_f,
                    'word_timestamp': word,
                    'segment_timestamp': new_text,
                })
                
    return new_data

In [9]:
r = loop((rows[:100], 0))
len(r)

100%|████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:02<00:00, 47.44it/s]


119

In [14]:
r[0]

{'audio_filename': '/home/husein/ssd3/prepared-imda-chunks/0-0.mp3',
 'word_timestamp': '<|startoftranscript|><|en|><|transcribeprecise|><|0.00|> Households<|0.58|><|0.64|> with<|0.76|><|0.86|> target<|1.16|><|1.24|> sets<|1.50|><|1.70|> were<|1.82|><|1.90|> encouraged<|2.40|><|2.44|> to<|2.48|><|2.62|> try<|2.80|><|2.90|> keeping<|3.30|><|endoftext|>',
 'segment_timestamp': '<|startoftranscript|><|en|><|transcribe|><|0.00|> Households with target sets were encouraged to try keeping<|3.30|><|endoftext|>'}

In [15]:
import IPython.display as ipd
ipd.Audio(r[0]['audio_filename'])

In [16]:
len(rows)

1861082

In [17]:
r = mp.multiprocessing(rows, loop, cores = 10)

100%|████████████████████████████████████████████████████████████████████████████████████████| 186108/186108 [2:31:07<00:00, 20.53it/s]


In [18]:
len(r)

2442094

In [19]:
r[-10]

{'audio_filename': '/home/husein/ssd3/prepared-imda-chunks/1861119-3.mp3',
 'word_timestamp': '<|startoftranscript|><|en|><|transcribeprecise|><|0.00|> a<|0.00|><|0.10|> feasible<|0.52|><|0.60|> way<|0.72|><|0.84|> for<|0.94|><|1.08|> us<|1.18|><|1.24|> to<|1.30|><|1.40|> pay<|1.58|><|1.88|> with<|2.12|><|endoftext|>',
 'segment_timestamp': '<|startoftranscript|><|en|><|transcribe|><|0.00|> a feasible way for us to pay<|1.58|><|1.88|> with<|2.12|><|endoftext|>'}

In [20]:
import copy
import re

results = []
for r_ in tqdm(r):
    r_ = copy.copy(r_)
    r_['audio_filename'] = r_['audio_filename'].replace('/home/husein/ssd3/', '').replace('/home/husein/ssd4/', '')
    clean_text = re.sub(r"<\|.*?\|>", "", r_['segment_timestamp']).strip().split()
    ratio = (len([w for w in clean_text if len(w) <= 1]) / len(clean_text))
    if ratio > 0.2:
        continue
    results.append(r_)

100%|████████████████████████████████████████████████████████████████████████████████████| 2442094/2442094 [00:06<00:00, 385960.71it/s]


In [21]:
from collections import defaultdict

def generate_trigrams(text):
    words = text.split()
    return list(zip(words, words[1:], words[2:]))

def skip_trigrams(text):
    trigrams = generate_trigrams(text)
    count = defaultdict(int)
    total = 0
    for t in trigrams:
        count[''.join(t)] += 1
        total += 1
    if len(count.keys()) < 3:
        return True
    for k, v in count.items():
        if (v / total) > 0.2:
            return True
    return False

In [22]:
filtered = []
for r_ in tqdm(results):
    if skip_trigrams(re.sub(r"<\|.*?\|>", "", r_['segment_timestamp']).strip()):
        continue
    filtered.append(r_)
len(filtered)

100%|████████████████████████████████████████████████████████████████████████████████████| 2289183/2289183 [00:08<00:00, 273625.84it/s]


1009338

In [24]:
len(filtered) / len(results)

0.440916257022702

In [25]:
import pandas as pd

pd.DataFrame(filtered).to_parquet('pseudolabel-imda-word-timestamp.parquet')

In [26]:
!ls -lh pseudolabel-imda-word-timestamp.parquet

-rw-rw-r-- 1 husein husein 173M Mac  20 16:07 pseudolabel-imda-word-timestamp.parquet


In [27]:
from huggingface_hub import HfApi
api = HfApi()
api.upload_file(
    path_or_fileobj="pseudolabel-imda-word-timestamp.parquet",
    path_in_repo="data/imda-00000-of-00001.parquet",
    repo_id="mesolitica/Malaysian-STT-Whisper",
    repo_type="dataset",
)

pseudolabel-imda-word-timestamp.parquet:   0%|          | 0.00/180M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/mesolitica/Malaysian-STT-Whisper/commit/39c5e06921d0f210eb1a78463d12690e92256d7c', commit_message='Upload data/imda-00000-of-00001.parquet with huggingface_hub', commit_description='', oid='39c5e06921d0f210eb1a78463d12690e92256d7c', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/mesolitica/Malaysian-STT-Whisper', endpoint='https://huggingface.co', repo_type='dataset', repo_id='mesolitica/Malaysian-STT-Whisper'), pr_revision=None, pr_num=None)